In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
# prevent ultralytics from tracking your activity
!yolo settings sync=False
import ultralytics
ultralytics.checks()

In [ ]:
!yolo task=detect mode=predict model=yolo11m-seg.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/segment/predict/dog.jpg', width=600)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11m-seg.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.25)[0]

In [ ]:
from roboflow import Roboflow

!roboflow workspace list

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("tbe") # Your workspace ID
project = workspace.project("rice-grain-svjri") # Your project ID
version = project.version(4)
dataset = version.download("yolov11")

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolo11m-seg.pt data={dataset.location}/data.yaml epochs=1000 imgsz=640 plots=True

In [ ]:
!ls {HOME}/runs/segment/train/

In [ ]:
# from IPython.display import Image as IPyImage

# IPyImage(filename=f'{HOME}/runs/segment/train/confusion_matrix.png', width=600)

In [ ]:
!yolo task=segment mode=val model={HOME}/runs/segment/train/weights/best.pt data={dataset.location}/data.yaml imgsz=640 plots=True max_det=1000

In [ ]:
# Data Augmentation

# Melakukan augmentasi pada dataset untuk meningkatkan variasi data training

In [ ]:
# Install library untuk augmentasi
%pip install albumentations opencv-python-headless

In [ ]:
import cv2
import numpy as np
import albumentations as A
from pathlib import Path
import shutil
from tqdm import tqdm
import yaml
from collections import Counter
import random

# Fungsi untuk membaca annotation YOLO format
def read_yolo_annotation(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = []
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) > 0:
                class_id = int(parts[0])
                coords = [float(x) for x in parts[1:]]
                annotations.append([class_id] + coords)
    return annotations

# Fungsi untuk menulis annotation YOLO format
def write_yolo_annotation(annotation_path, annotations):
    with open(annotation_path, 'w') as f:
        for ann in annotations:
            class_id = int(ann[0])
            coords = ' '.join([f'{x:.6f}' for x in ann[1:]])
            f.write(f'{class_id} {coords}\n')

# Fungsi untuk membaca konfigurasi dataset dari YAML
def load_data_config(data_yaml_path, dataset_path_hint=None):
    data_yaml_path = Path(data_yaml_path)
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    names = data.get('names', [])
    if isinstance(names, dict):
        names = [names[str(i)] for i in range(len(names))]

    # Gunakan dataset_path_hint sebagai base jika diberikan, jika tidak gunakan parent dari data.yaml
    if dataset_path_hint:
        base_dir = Path(dataset_path_hint).resolve()
    else:
        base_dir = data_yaml_path.parent.resolve()

    def resolve_path(path_value: str | Path):
        path_value = Path(path_value)
        # Jika absolut, kembalikan langsung
        if path_value.is_absolute():
            return path_value.resolve()
        # Jika relatif, gabungkan dengan base_dir dan resolve
        # Normalisasi: hilangkan .. dan . dari path
        parts = []
        for part in path_value.parts:
            if part == '..':
                if parts:
                    parts.pop()
            elif part != '.':
                parts.append(part)
        normalized = Path(*parts) if parts else Path('.')
        resolved = (base_dir / normalized).resolve()
        return resolved

    splits = {}
    for split_key in ('train', 'val', 'test'):
        split_path = data.get(split_key)
        if not split_path:
            continue
        images_dir = resolve_path(split_path)
        # Labels biasanya sejajar dengan images di parent/labels
        labels_dir = (images_dir.parent / 'labels').resolve()
        splits[split_key] = {
            'images': images_dir,
            'labels': labels_dir,
        }

    return {
        'names': names,
        'splits': splits,
    }

# Menghitung distribusi kelas dan statistik per gambar
def collect_class_stats(labels_dir, num_classes):
    labels_dir = Path(labels_dir)
    counts = Counter({cls: 0 for cls in range(num_classes)})
    image_stats = {}

    for label_path in labels_dir.glob('*.txt'):
        annotations = read_yolo_annotation(label_path)
        per_image = Counter({cls: 0 for cls in range(num_classes)})
        for ann in annotations:
            class_id = int(ann[0])
            if class_id >= num_classes:
                continue
            per_image[class_id] += 1
            counts[class_id] += 1
        image_stats[label_path.stem] = per_image

    return counts, image_stats

# Menampilkan distribusi kelas yang mudah dibaca
def print_class_distribution(split_name, counts, class_names):
    total = sum(counts.values())
    print(f"\nDistribusi kelas untuk {split_name}:")
    for idx, class_name in enumerate(class_names):
        value = counts.get(idx, 0)
        if total > 0:
            pct = (value / total) * 100
            print(f"  - {class_name}: {value} ({pct:.2f}%)")
        else:
            print(f"  - {class_name}: {value}")

# Menentukan rencana augmentasi agar kelas minoritas mendekati jumlah kelas mayoritas
def build_balanced_augmentation_plan(image_stats, target_class_id, deficit, max_aug_per_image=5):
    plan = {}
    if deficit <= 0:
        return plan

    eligible = []
    for stem, stats in image_stats.items():
        instances = stats.get(target_class_id, 0)
        if instances > 0:
            eligible.append((stem, instances))

    if not eligible:
        return plan

    eligible.sort(key=lambda item: item[1], reverse=True)
    total_capacity = sum(instances * max_aug_per_image for _, instances in eligible)
    if total_capacity < deficit:
        print(
            f"Peringatan: kapasitas augmentasi maksimum ({total_capacity}) lebih kecil dari kebutuhan ({deficit}). "
            "Dataset mungkin tetap tidak seimbang."
        )

    idx = 0
    iterations = 0
    max_iterations = len(eligible) * max_aug_per_image if eligible else 0

    while deficit > 0 and idx < max_iterations and eligible:
        stem, instances = eligible[idx % len(eligible)]
        if plan.get(stem, 0) >= max_aug_per_image:
            idx += 1
            iterations += 1
            continue

        plan[stem] = plan.get(stem, 0) + 1
        deficit -= instances
        idx += 1
        iterations += 1

    if deficit > 0:
        print(f"Peringatan: Masih ada selisih {deficit} instance setelah perencanaan augmentasi.")

    return plan

# Mengambil patch objek dari polygon YOLO (segmentation)
def extract_object_patch(image, polygon_coords, padding=2):
    height, width = image.shape[:2]
    if len(polygon_coords) < 6:
        return None

    pts = np.array(polygon_coords, dtype=np.float32).reshape(-1, 2)
    x_px = np.clip(np.round(pts[:, 0] * width), 0, width - 1)
    y_px = np.clip(np.round(pts[:, 1] * height), 0, height - 1)
    pts_px = np.stack([x_px, y_px], axis=1).astype(np.int32)

    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, [pts_px], 1)

    x_min = max(0, int(np.min(pts_px[:, 0])) - padding)
    x_max = min(width, int(np.max(pts_px[:, 0])) + padding)
    y_min = max(0, int(np.min(pts_px[:, 1])) - padding)
    y_max = min(height, int(np.max(pts_px[:, 1])) + padding)

    if x_max - x_min < 2 or y_max - y_min < 2:
        return None

    patch = image[y_min:y_max, x_min:x_max]
    mask_patch = mask[y_min:y_max, x_min:x_max]

    if mask_patch.max() == 0:
        return None

    polygon_local = pts_px - np.array([x_min, y_min], dtype=np.int32)
    return patch, mask_patch, polygon_local

# Menempelkan patch pada gambar target dan mengembalikan polygon baru
def paste_patch_on_base(base_image, patch, mask_patch, polygon_local, sigma=3):
    base_height, base_width = base_image.shape[:2]
    patch_height, patch_width = patch.shape[:2]

    if patch_height == 0 or patch_width == 0:
        return None

    if patch_height > base_height or patch_width > base_width:
        return None

    max_x = base_width - patch_width
    max_y = base_height - patch_height

    if max_x < 0 or max_y < 0:
        return None

    if max_x == 0 and max_y == 0:
        x_offset, y_offset = 0, 0
    else:
        x_offset = random.randint(0, max_x)
        y_offset = random.randint(0, max_y)

    mask_float = mask_patch.astype(np.float32)
    if mask_float.max() == 0:
        return None
    mask_float /= mask_float.max()

    if sigma and sigma > 0:
        mask_float = cv2.GaussianBlur(mask_float, (0, 0), sigmaX=sigma, sigmaY=sigma)
    mask_float = np.clip(mask_float, 0.0, 1.0)
    mask_float = mask_float[..., None]

    roi = base_image[y_offset:y_offset + patch_height, x_offset:x_offset + patch_width]
    blended = (mask_float * patch.astype(np.float32) + (1.0 - mask_float) * roi.astype(np.float32)).astype(np.uint8)
    base_image[y_offset:y_offset + patch_height, x_offset:x_offset + patch_width] = blended

    polygon_shifted = polygon_local + np.array([x_offset, y_offset], dtype=np.int32)
    polygon_norm = []
    for x_px, y_px in polygon_shifted:
        polygon_norm.append(float(np.clip(x_px / base_width, 0.0, 1.0)))
        polygon_norm.append(float(np.clip(y_px / base_height, 0.0, 1.0)))

    return polygon_norm

print("Fungsi utilitas augmentasi copy-paste berhasil didefinisikan!")

In [ ]:
# Transformasi warna/pixel opsional setelah copy-paste
import albumentations as A

transform_color = A.Compose([
    A.RandomBrightnessContrast(
        brightness_limit=0.2, 
        contrast_limit=0.2, 
        p=0.5
    ),
    A.HueSaturationValue(
        hue_shift_limit=10,
        sat_shift_limit=20,
        val_shift_limit=10,
        p=0.5
    ),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.CLAHE(clip_limit=2.0, p=0.3),
], bbox_params=None)

print("Pipeline augmentasi warna berhasil didefinisikan (opsional setelah copy-paste)!")
print("Transformasi warna/pixel yang digunakan:")
print("  1. RandomBrightnessContrast (50%)")
print("  2. HueSaturationValue (50%)")
print("  3. GaussNoise (30%)")
print("  4. GaussianBlur (30%)")
print("  5. CLAHE (30%)")

In [ ]:
# Visualisasi hasil augmentasi
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
from pathlib import Path

def visualize_augmentations(images_path, num_samples=4):
    """Visualisasi gambar original dan hasil augmentasinya."""
    images_dir = Path(images_path)
    if not images_dir.exists():
        print(f"Direktori gambar tidak ditemukan: {images_dir}")
        print("Pastikan cell konfigurasi berjalan dan augmentasi menghasilkan gambar.")
        return

    # Filter: gambar original adalah yang TIDAK mengandung '_cp_' atau '_aug' di filename
    original_images = [
        f.name
        for f in images_dir.iterdir()
        if f.is_file() 
        and '_cp_' not in f.stem 
        and '_aug' not in f.stem 
        and f.suffix.lower() in {'.jpg', '.jpeg', '.png'}
    ]

    if len(original_images) == 0:
        print("Tidak ada gambar original ditemukan")
        return

    sample_images = random.sample(original_images, min(num_samples, len(original_images)))

    for orig_name in sample_images:
        base_name = Path(orig_name).stem
        # Cari gambar copy-paste yang berasal dari gambar original ini
        # Format: {base_name}_cp_*
        aug_images = [
            f.name
            for f in images_dir.iterdir()
            if f.is_file() 
            and f.name.startswith(base_name + '_cp_') 
            and f.suffix.lower() in {'.jpg', '.jpeg', '.png'}
        ]

        if len(aug_images) == 0:
            print(f"Tidak ada hasil copy-paste untuk {orig_name}.")
            continue

        num_cols = min(4, len(aug_images) + 1)
        fig, axes = plt.subplots(1, num_cols, figsize=(20, 5))

        if num_cols == 1:
            axes = [axes]

        orig_img = Image.open(images_dir / orig_name)
        axes[0].imshow(orig_img)
        axes[0].set_title('Original')
        axes[0].axis('off')

        for idx, aug_name in enumerate(aug_images[:num_cols-1]):
            aug_img = Image.open(images_dir / aug_name)
            axes[idx+1].imshow(aug_img)
            axes[idx+1].set_title(f'Copy-Paste {idx+1}')
            axes[idx+1].axis('off')

        plt.tight_layout()
        plt.show()
        print(f"Gambar: {orig_name}")
        print(f"  - Jumlah augmentasi: {len(aug_images)}")
        print("-" * 50)

# NOTE: pemanggilan visualize_augmentations dilakukan setelah proses augmentasi selesai

In [ ]:
# Konfigurasi path dan parameter copy-paste balancing
from pathlib import Path

# Gunakan path dataset dari hasil download Roboflow jika tersedia
if 'dataset' in globals():
    DATASET_PATH = Path(dataset.location).resolve()
else:
    DATASET_PATH = Path("/content/datasets/Rice-Grain-4").resolve()

DATA_CONFIG_PATH = (DATASET_PATH / "data.yaml").resolve()
if not DATA_CONFIG_PATH.exists():
    raise FileNotFoundError(f"data.yaml tidak ditemukan di {DATA_CONFIG_PATH}. Pastikan dataset sudah diunduh.")

data_config = load_data_config(DATA_CONFIG_PATH, dataset_path_hint=DATASET_PATH)
CLASS_NAMES = data_config["names"]
SPLITS = data_config["splits"]

if "train" not in SPLITS:
    raise ValueError("Path train tidak ditemukan pada data.yaml. Pastikan konfigurasi dataset benar.")

TRAIN_IMAGES_PATH = SPLITS["train"]["images"]
TRAIN_LABELS_PATH = SPLITS["train"]["labels"]
VAL_IMAGES_PATH = SPLITS.get("val", {}).get("images")
VAL_LABELS_PATH = SPLITS.get("val", {}).get("labels")
TEST_IMAGES_PATH = SPLITS.get("test", {}).get("images")
TEST_LABELS_PATH = SPLITS.get("test", {}).get("labels")

# Verifikasi path yang dihasilkan
for name, path_value in [
    ("TRAIN_IMAGES_PATH", TRAIN_IMAGES_PATH),
    ("TRAIN_LABELS_PATH", TRAIN_LABELS_PATH),
    ("VAL_IMAGES_PATH", VAL_IMAGES_PATH),
    ("VAL_LABELS_PATH", VAL_LABELS_PATH),
    ("TEST_IMAGES_PATH", TEST_IMAGES_PATH),
    ("TEST_LABELS_PATH", TEST_LABELS_PATH),
]:
    if path_value is None:
        continue
    if not path_value.exists():
        print(f"Peringatan: {name} tidak ditemukan di {path_value}")

# Parameter balancing berbasis copy-paste
TARGET_MINORITY_CLASS_NAME = "brown_spot"
DEFAULT_BASE_AUGMENTATIONS = 0      # augmentasi dasar untuk semua gambar
MAX_AUG_PER_IMAGE = 5               # batas augmentasi tambahan per gambar minoritas
COPY_PASTE_MIN_OBJECTS = 1          # minimal objek minoritas yang ditempel per gambar baru
COPY_PASTE_MAX_OBJECTS = 3          # maksimal objek minoritas yang ditempel per gambar baru
COPY_PASTE_PADDING = 4              # padding di sekitar mask saat memotong objek
MASK_BLUR_SIGMA = 3                 # smoothing tepi saat penempelan
APPLY_COLOR_AUG = True              # gunakan transformasi warna setelah copy-paste

if COPY_PASTE_MAX_OBJECTS < COPY_PASTE_MIN_OBJECTS:
    raise ValueError("COPY_PASTE_MAX_OBJECTS harus >= COPY_PASTE_MIN_OBJECTS")

print(f"Dataset path: {DATASET_PATH}")
print(f"Train images path: {TRAIN_IMAGES_PATH}")
print(f"Train labels path: {TRAIN_LABELS_PATH}")
if VAL_IMAGES_PATH:
    print(f"Validation images path: {VAL_IMAGES_PATH}")
if TEST_IMAGES_PATH:
    print(f"Test images path: {TEST_IMAGES_PATH}")
print(f"Jumlah kelas: {len(CLASS_NAMES)} -> {CLASS_NAMES}")
print("Parameter copy-paste:")
print(f"  - Target kelas minoritas : {TARGET_MINORITY_CLASS_NAME}")
print(f"  - Min objek per augmentasi: {COPY_PASTE_MIN_OBJECTS}")
print(f"  - Max objek per augmentasi: {COPY_PASTE_MAX_OBJECTS}")
print(f"  - Padding objek          : {COPY_PASTE_PADDING}")
print(f"  - Mask blur sigma        : {MASK_BLUR_SIGMA}")
print(f"  - Color augment aktif    : {APPLY_COLOR_AUG}")

In [ ]:
# Analisis distribusi kelas & rencana balancing
NUM_CLASSES = len(CLASS_NAMES)

train_counts, TRAIN_IMAGE_STATS = collect_class_stats(TRAIN_LABELS_PATH, NUM_CLASSES)
val_counts = Counter({cls: 0 for cls in range(NUM_CLASSES)})
test_counts = Counter({cls: 0 for cls in range(NUM_CLASSES)})

if VAL_LABELS_PATH:
    val_counts, _ = collect_class_stats(VAL_LABELS_PATH, NUM_CLASSES)
if TEST_LABELS_PATH:
    test_counts, _ = collect_class_stats(TEST_LABELS_PATH, NUM_CLASSES)

print_class_distribution("Train (sebelum augmentasi)", train_counts, CLASS_NAMES)
if VAL_LABELS_PATH:
    print_class_distribution("Validation", val_counts, CLASS_NAMES)
if TEST_LABELS_PATH:
    print_class_distribution("Test", test_counts, CLASS_NAMES)

if TARGET_MINORITY_CLASS_NAME in CLASS_NAMES:
    minority_class_id = CLASS_NAMES.index(TARGET_MINORITY_CLASS_NAME)
else:
    minority_class_id = min(train_counts, key=train_counts.get)
majority_class_id = max(train_counts, key=train_counts.get)

balance_deficit = train_counts[majority_class_id] - train_counts[minority_class_id]

print("\nRingkasan balancing:")
print(f"Kelas mayoritas : {CLASS_NAMES[majority_class_id]} ({train_counts[majority_class_id]} instance)")
print(f"Kelas minoritas : {CLASS_NAMES[minority_class_id]} ({train_counts[minority_class_id]} instance)")
print(f"Selisih instance : {balance_deficit}")

AUGMENTATION_PLAN = build_balanced_augmentation_plan(
    TRAIN_IMAGE_STATS,
    target_class_id=minority_class_id,
    deficit=balance_deficit,
    max_aug_per_image=MAX_AUG_PER_IMAGE,
)

expected_new_minority = sum(
    TRAIN_IMAGE_STATS[stem].get(minority_class_id, 0) * count
    for stem, count in AUGMENTATION_PLAN.items()
)

print(f"\nRencana augmentasi untuk kelas {CLASS_NAMES[minority_class_id]}:")
print(f"  - Jumlah gambar unik yang ditambah: {len(AUGMENTATION_PLAN)}")
print(f"  - Total augmentasi baru: {sum(AUGMENTATION_PLAN.values())}")
print(f"  - Perkiraan penambahan instance minoritas: {expected_new_minority}")
if balance_deficit > 0 and not AUGMENTATION_PLAN:
    print("  ! Tidak ada gambar minoritas yang tersedia untuk di-augment. Dataset tetap tidak seimbang.")

BASELINE_TRAIN_COUNTS = train_counts.copy()
MINORITY_CLASS_ID = minority_class_id
MAJORITY_CLASS_ID = majority_class_id

In [ ]:
# Fungsi untuk melakukan augmentasi copy-paste pada segmentation dataset
def augment_segmentation_dataset(
    images_path,
    labels_path,
    base_num_augmentations=0,
    augmentation_plan=None,
    minority_class_id=0,
    class_names=None,
    apply_color_aug=False,
    mask_blur_sigma=3,
    min_objects_per_paste=1,
    max_objects_per_paste=3,
    padding=4,
):
    """Generate balanced data using copy-paste augmentation for YOLO segmentation."""

    images_dir = Path(images_path)
    labels_dir = Path(labels_path)
    augmentation_plan = augmentation_plan or {}

    image_files = sorted(list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.jpeg')) + list(images_dir.glob('*.png')))
    if not image_files:
        print("Tidak ada gambar ditemukan di direktori train.")
        return 0

    stem_to_path = {img_path.stem: img_path for img_path in image_files}
    label_exists = {stem: (labels_dir / f"{stem}.txt").exists() for stem in stem_to_path}
    base_stems = [stem for stem, exists in label_exists.items() if exists]

    if not base_stems:
        print("Tidak ada file label YOLO yang ditemukan. Augmentasi dibatalkan.")
        return 0

    class_names = class_names or []

    augmented_images = 0
    total_pasted_instances = 0

    for stem, donor_img_path in stem_to_path.items():
        total_aug = base_num_augmentations + augmentation_plan.get(stem, 0)
        if total_aug <= 0:
            continue

        donor_label_path = labels_dir / f"{stem}.txt"
        if not donor_label_path.exists():
            continue

        donor_annotations = read_yolo_annotation(donor_label_path)
        minority_annotations = [ann for ann in donor_annotations if int(ann[0]) == minority_class_id]
        if not minority_annotations:
            continue

        donor_bgr = cv2.imread(str(donor_img_path))
        if donor_bgr is None:
            print(f"Gagal membaca gambar donor: {donor_img_path}")
            continue
        donor_rgb = cv2.cvtColor(donor_bgr, cv2.COLOR_BGR2RGB)

        for aug_idx in range(total_aug):
            base_stem = random.choice(base_stems)
            base_img_path = stem_to_path[base_stem]
            base_label_path = labels_dir / f"{base_stem}.txt"

            base_annotations = read_yolo_annotation(base_label_path)
            base_bgr = cv2.imread(str(base_img_path))
            if base_bgr is None:
                print(f"Gagal membaca gambar target: {base_img_path}")
                continue
            base_rgb = cv2.cvtColor(base_bgr, cv2.COLOR_BGR2RGB)

            new_image = base_rgb.copy()
            new_annotations = [list(ann) for ann in base_annotations]
            pasted_this_image = 0

            max_pick = min(max_objects_per_paste, len(minority_annotations))
            min_pick = min(min_objects_per_paste, max_pick)
            if max_pick <= 0 or min_pick <= 0:
                continue

            num_to_paste = random.randint(min_pick, max_pick)
            selected_objects = random.sample(minority_annotations, num_to_paste)

            for obj in selected_objects:
                result = extract_object_patch(donor_rgb, obj[1:], padding=padding)
                if result is None:
                    continue
                patch, mask_patch, polygon_local = result
                new_polygon = paste_patch_on_base(
                    new_image,
                    patch,
                    mask_patch,
                    polygon_local,
                    sigma=mask_blur_sigma,
                )
                if new_polygon is None:
                    continue
                new_annotations.append([obj[0]] + new_polygon)
                pasted_this_image += 1
                total_pasted_instances += 1

            if pasted_this_image == 0:
                continue

            if apply_color_aug:
                augmented = transform_color(image=new_image)
                new_image = augmented['image']

            base_suffix = base_img_path.suffix or '.jpg'
            new_stem = f"{base_stem}_cp_{stem}_{aug_idx}"
            unique_id = 0
            new_image_path = images_dir / f"{new_stem}{base_suffix}"
            new_label_path = labels_dir / f"{new_stem}.txt"

            while new_image_path.exists() or new_label_path.exists():
                unique_id += 1
                new_stem = f"{base_stem}_cp_{stem}_{aug_idx}_{unique_id}"
                new_image_path = images_dir / f"{new_stem}{base_suffix}"
                new_label_path = labels_dir / f"{new_stem}.txt"

            cv2.imwrite(str(new_image_path), cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR))
            write_yolo_annotation(new_label_path, new_annotations)
            augmented_images += 1

    class_name = class_names[minority_class_id] if class_names and minority_class_id < len(class_names) else minority_class_id
    print(
        f"\nSelesai! Total {augmented_images} gambar baru telah dibuat dengan copy-paste."
    )
    print(
        f"Total instance kelas {class_name} yang ditempel: {total_pasted_instances}"
    )

    return augmented_images

print("Fungsi augment_segmentation_dataset copy-paste berhasil didefinisikan!")

In [ ]:
# Jalankan augmentasi copy-paste
print("Memulai proses augmentasi copy-paste dengan balancing kelas...")
print("=" * 50)

augmented_count = augment_segmentation_dataset(
    images_path=TRAIN_IMAGES_PATH,
    labels_path=TRAIN_LABELS_PATH,
    base_num_augmentations=DEFAULT_BASE_AUGMENTATIONS,
    augmentation_plan=AUGMENTATION_PLAN,
    minority_class_id=MINORITY_CLASS_ID,
    class_names=CLASS_NAMES,
    apply_color_aug=APPLY_COLOR_AUG,
    mask_blur_sigma=MASK_BLUR_SIGMA,
    min_objects_per_paste=COPY_PASTE_MIN_OBJECTS,
    max_objects_per_paste=COPY_PASTE_MAX_OBJECTS,
    padding=COPY_PASTE_PADDING,
)

print("=" * 50)
print("Augmentasi selesai!")
print(f"Total gambar baru: {augmented_count}")

In [ ]:
# Verifikasi hasil augmentasi dan balancing
train_counts_after, _ = collect_class_stats(TRAIN_LABELS_PATH, len(CLASS_NAMES))

if VAL_LABELS_PATH:
    val_counts_after, _ = collect_class_stats(VAL_LABELS_PATH, len(CLASS_NAMES))
else:
    val_counts_after = Counter({cls: 0 for cls in range(len(CLASS_NAMES))})

if TEST_LABELS_PATH:
    test_counts_after, _ = collect_class_stats(TEST_LABELS_PATH, len(CLASS_NAMES))
else:
    test_counts_after = Counter({cls: 0 for cls in range(len(CLASS_NAMES))})

print("Statistik Dataset Setelah Augmentasi:")
print("=" * 50)
print(f"Path dataset: {DATASET_PATH}")
print(f"Train images path: {TRAIN_IMAGES_PATH}")
print(f"Train labels path: {TRAIN_LABELS_PATH}")

if 'BASELINE_TRAIN_COUNTS' in globals():
    print("\nPerbandingan distribusi kelas (train):")
    for idx, class_name in enumerate(CLASS_NAMES):
        before = BASELINE_TRAIN_COUNTS.get(idx, 0)
        after = train_counts_after.get(idx, 0)
        delta = after - before
        sign = "+" if delta >= 0 else ""
        print(f"  - {class_name}: sebelum={before}, sesudah={after} ({sign}{delta})")
else:
    print_class_distribution("Train (setelah augmentasi)", train_counts_after, CLASS_NAMES)

if VAL_LABELS_PATH:
    print_class_distribution("Validation", val_counts_after, CLASS_NAMES)
if TEST_LABELS_PATH:
    print_class_distribution("Test", test_counts_after, CLASS_NAMES)

print("\nContoh file hasil augmentasi:")
aug_files = [
    f for f in sorted(TRAIN_IMAGES_PATH.glob('*'))
    if '_aug' in f.stem and f.suffix.lower() in {'.jpg', '.jpeg', '.png'}
]
for i, path in enumerate(aug_files[:5]):
    print(f"  {i+1}. {path.name}")

In [ ]:
# Tampilkan visualisasi hasil augmentasi
visualize_augmentations(TRAIN_IMAGES_PATH, num_samples=2)